# Heckman MLE Estimation -- SOLUTION

**This is the worked solution notebook.**  
It mirrors `05_heckman_mle.ipynb` and fills in all exercise cells with complete, working solutions.

> Instructors: do not distribute this file to students before they complete the tutorial notebook.

**Date**: 2026-02-17

---

## Exercises Covered

1. **Exercise 1** -- Log-Likelihood Computation (Easy)
2. **Exercise 2** -- Sensitivity to Exclusion Restrictions (Medium)
3. **Exercise 3** -- Convergence Diagnostics (Medium)
4. **Exercise 4** -- Monte Carlo Comparison (Hard)

## Setup

Import all required libraries and configure the environment.

In [ ]:
import sys
import pathlib

ROOT = pathlib.Path('..').resolve()
PANELBOX_ROOT = pathlib.Path('/home/guhaase/projetos/panelbox')
for p in [str(ROOT), str(PANELBOX_ROOT)]:
    if p not in sys.path:
        sys.path.insert(0, p)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.optimize import minimize
import statsmodels.api as sm

from panelbox.models.selection import PanelHeckman

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

np.random.seed(42)

BASE_DIR = pathlib.Path('..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = BASE_DIR / 'outputs'
FIGURES_DIR = OUTPUT_DIR / 'figures'
TABLES_DIR = OUTPUT_DIR / 'tables'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print('Solution notebook loaded.')

## Data Loading

Load the Mroz (1987) dataset and prepare outcome/selection variables.

In [ ]:
# Load the Mroz 1987 dataset
data = pd.read_csv(DATA_DIR / 'mroz_1987.csv')

print(f"Dataset shape: {data.shape}")
print(f"Columns: {list(data.columns)}")
print(f"\nParticipation rate: {data['lfp'].mean():.2%}")
print(f"Participants (lfp=1): {data['lfp'].sum()}")
print(f"Non-participants (lfp=0): {(1 - data['lfp']).sum():.0f}")
print(f"\nWage statistics (participants only):")
print(data.loc[data['lfp'] == 1, 'wage'].describe())

In [ ]:
# Prepare variables for estimation

# Selection indicator
selection = data['lfp'].values

# Outcome variable: wage (use 0 for non-participants)
wage = data['wage'].fillna(0).values

# Outcome equation regressors: const, education, experience, experience_sq
X = sm.add_constant(
    data[['education', 'experience', 'experience_sq']].values
)
x_names = ['const', 'education', 'experience', 'experience_sq']

# Selection equation regressors: const, education, experience, age,
#   children_lt6, children_6_18, husband_income
Z = sm.add_constant(
    data[['education', 'experience', 'age',
          'children_lt6', 'children_6_18', 'husband_income']].values
)
z_names = ['const', 'education', 'experience', 'age',
           'children_lt6', 'children_6_18', 'husband_income']

print(f"Outcome regressors (X): {x_names}")
print(f"Selection regressors (Z): {z_names}")
print(f"\nX shape: {X.shape}")
print(f"Z shape: {Z.shape}")
print(f"\nExclusion restrictions: age, children_lt6, children_6_18, husband_income")

## Fit Base Models for Reference

Estimate both two-step and MLE Heckman models as baselines for the exercises.

In [ ]:
# Two-step estimation
model_2s = PanelHeckman(
    endog=wage,
    exog=X,
    selection=selection,
    exog_selection=Z,
    method='two_step'
)
result_2s = model_2s.fit()

print("=" * 65)
print("  TWO-STEP HECKMAN ESTIMATES (Baseline)")
print("=" * 65)
print(f"\n{'Variable':<20} {'Coefficient':>12}")
print("-" * 34)
for name, coef in zip(x_names, result_2s.outcome_params):
    print(f"{name:<20} {coef:>12.4f}")
print(f"\nsigma = {result_2s.sigma:.4f}")
print(f"rho   = {result_2s.rho:.4f}")
print(f"lambda = {result_2s.rho * result_2s.sigma:.4f}")

In [ ]:
# MLE estimation
model_ml = PanelHeckman(
    endog=wage,
    exog=X,
    selection=selection,
    exog_selection=Z,
    method='mle'
)
result_ml = model_ml.fit()

print("=" * 65)
print("  MLE HECKMAN ESTIMATES (Baseline)")
print("=" * 65)
print(f"\n{'Variable':<20} {'Coefficient':>12}")
print("-" * 34)
for name, coef in zip(x_names, result_ml.outcome_params):
    print(f"{name:<20} {coef:>12.4f}")
print(f"\nsigma     = {result_ml.sigma:.4f}")
print(f"rho       = {result_ml.rho:.4f}")
print(f"lambda    = {result_ml.rho * result_ml.sigma:.4f}")
if result_ml.llf is not None:
    print(f"Log-lik   = {result_ml.llf:.4f}")
print(f"Converged = {result_ml.converged}")

---

## Exercise 1: Log-Likelihood Computation (Easy)

**Task**: Manually compute the log-likelihood contribution for individual observations using the MLE estimates.

For a **selected** observation ($s_i = 1$):

$$\ell_i^{\text{sel}} = \log\phi\left(\frac{y_i - X_i'\beta}{\sigma}\right) - \log\sigma + \log\Phi(z_i^*)$$

where $z_i^* = \frac{Z_i'\gamma + \rho(y_i - X_i'\beta)/\sigma}{\sqrt{1 - \rho^2}}$

For a **non-selected** observation ($s_i = 0$):

$$\ell_i^{\text{non}} = \log\Phi(-Z_i'\gamma)$$

In [ ]:
# ============================================================
# EXERCISE 1 SOLUTION: Log-Likelihood Computation
# ============================================================

# Step 1: Extract MLE parameter estimates
beta_hat = result_ml.outcome_params
gamma_hat = result_ml.probit_params
sigma_hat = result_ml.sigma
rho_hat = result_ml.rho

print("MLE Parameter Estimates:")
print(f"  beta  = {beta_hat}")
print(f"  gamma = {gamma_hat}")
print(f"  sigma = {sigma_hat:.6f}")
print(f"  rho   = {rho_hat:.6f}")

# -------------------------------------------------------
# Part A: Selected observation (s_i = 1)
# -------------------------------------------------------
first_selected_idx = np.where(selection == 1)[0][0]
y_i = wage[first_selected_idx]
X_i = X[first_selected_idx]
Z_i = Z[first_selected_idx]

print(f"\n--- Selected Observation (index = {first_selected_idx}) ---")
print(f"  y_i (wage)  = {y_i:.4f}")
print(f"  X_i         = {X_i}")
print(f"  Z_i         = {Z_i}")

# Compute the linear predictions
Xb_i = X_i @ beta_hat
Zg_i = Z_i @ gamma_hat
print(f"\n  X_i'beta    = {Xb_i:.6f}")
print(f"  Z_i'gamma   = {Zg_i:.6f}")

# (1) Standardized residual
residual_i = (y_i - Xb_i) / sigma_hat
print(f"\n  Standardized residual = (y_i - X_i'beta) / sigma")
print(f"                       = ({y_i:.4f} - {Xb_i:.4f}) / {sigma_hat:.4f}")
print(f"                       = {residual_i:.6f}")

# (2) Adjusted selection index z*
z_star_i = (Zg_i + rho_hat * residual_i) / np.sqrt(1 - rho_hat**2)
print(f"\n  z_i* = (Z_i'gamma + rho * residual) / sqrt(1 - rho^2)")
print(f"       = ({Zg_i:.4f} + {rho_hat:.4f} * {residual_i:.4f}) / sqrt(1 - {rho_hat:.4f}^2)")
print(f"       = {z_star_i:.6f}")

# (3) Full log-likelihood contribution for selected observation
# ell_i = log(phi(residual)) - log(sigma) + log(Phi(z*))
# Note: log(phi(x)) = -0.5*log(2*pi) - 0.5*x^2
log_phi_term = np.log(stats.norm.pdf(residual_i))
log_sigma_term = -np.log(sigma_hat)
log_Phi_term = np.log(stats.norm.cdf(z_star_i))

ell_selected = log_phi_term + log_sigma_term + log_Phi_term

print(f"\n  Log-likelihood contribution (selected):")
print(f"    log phi(residual)  = {log_phi_term:.6f}")
print(f"    - log(sigma)       = {log_sigma_term:.6f}")
print(f"    log Phi(z*)        = {log_Phi_term:.6f}")
print(f"    -----------------------------------")
print(f"    ell_i (total)      = {ell_selected:.6f}")

# -------------------------------------------------------
# Part B: Non-selected observation (s_i = 0)
# -------------------------------------------------------
first_nonselected_idx = np.where(selection == 0)[0][0]
Z_j = Z[first_nonselected_idx]

print(f"\n--- Non-Selected Observation (index = {first_nonselected_idx}) ---")
print(f"  Z_j         = {Z_j}")

Zg_j = Z_j @ gamma_hat
print(f"  Z_j'gamma   = {Zg_j:.6f}")

# Non-selected contribution: log(Phi(-Z'gamma))
ell_nonselected = np.log(stats.norm.cdf(-Zg_j))

print(f"\n  Log-likelihood contribution (non-selected):")
print(f"    log Phi(-Z_j'gamma) = log Phi({-Zg_j:.4f})")
print(f"                        = {ell_nonselected:.6f}")

# -------------------------------------------------------
# Verification: compare manual sum vs model log-likelihood
# -------------------------------------------------------
print("\n" + "=" * 60)
print("  VERIFICATION: Manual full log-likelihood")
print("=" * 60)

total_ll_manual = 0.0
Xb_all = X @ beta_hat
Zg_all = Z @ gamma_hat

for i in range(len(selection)):
    if selection[i] == 1:
        res_i = (wage[i] - Xb_all[i]) / sigma_hat
        zs_i = (Zg_all[i] + rho_hat * res_i) / np.sqrt(1 - rho_hat**2)
        total_ll_manual += (
            np.log(stats.norm.pdf(res_i))
            - np.log(sigma_hat)
            + np.log(stats.norm.cdf(zs_i))
        )
    else:
        total_ll_manual += np.log(stats.norm.cdf(-Zg_all[i]))

print(f"\n  Manual log-likelihood sum:  {total_ll_manual:.6f}")
if result_ml.llf is not None:
    print(f"  Model reported llf:        {result_ml.llf:.6f}")
    print(f"  Difference:                {abs(total_ll_manual - result_ml.llf):.2e}")
    if abs(total_ll_manual - result_ml.llf) < 1e-3:
        print("  --> Manual computation matches the model log-likelihood.")
    else:
        print("  --> Small numerical difference (expected due to floating-point precision).")

---

## Exercise 2: Sensitivity to Exclusion Restrictions (Medium)

**Task**: Investigate how the choice of exclusion restrictions affects MLE estimates.

1. Estimate with only `husband_income` as the exclusion restriction
2. Estimate with all four exclusion restrictions (baseline)
3. Compare $\hat{\rho}$, $\hat{\sigma}$, and the outcome coefficients

In [ ]:
# ============================================================
# EXERCISE 2 SOLUTION: Sensitivity to Exclusion Restrictions
# ============================================================

# -------------------------------------------------------
# Specification 1: Minimal exclusion (husband_income only)
# -------------------------------------------------------
# Selection equation: const, education, experience, husband_income
# The only variable in Z but not in X is husband_income.
Z_minimal = sm.add_constant(
    data[['education', 'experience', 'husband_income']].values
)
z_names_minimal = ['const', 'education', 'experience', 'husband_income']

model_minimal = PanelHeckman(
    endog=wage,
    exog=X,
    selection=selection,
    exog_selection=Z_minimal,
    method='mle'
)
result_minimal = model_minimal.fit()

print("Specification 1: Minimal exclusion (husband_income only)")
print(f"  Converged: {result_minimal.converged}")
print(f"  rho   = {result_minimal.rho:.4f}")
print(f"  sigma = {result_minimal.sigma:.4f}")

# -------------------------------------------------------
# Specification 2: Full exclusion restrictions (baseline)
# -------------------------------------------------------
# Already estimated as result_ml
print(f"\nSpecification 2: Full exclusion (age, children_lt6, children_6_18, husband_income)")
print(f"  Converged: {result_ml.converged}")
print(f"  rho   = {result_ml.rho:.4f}")
print(f"  sigma = {result_ml.sigma:.4f}")

In [ ]:
# -------------------------------------------------------
# Comparison table
# -------------------------------------------------------
print("\n" + "=" * 75)
print("  EXCLUSION RESTRICTION SENSITIVITY ANALYSIS")
print("=" * 75)

# Outcome equation comparison
print("\n--- Outcome Equation ---")
print(f"{'Variable':<18} {'Minimal (1 excl)':>16} {'Full (4 excl)':>16} {'Difference':>12}")
print("-" * 64)
for i, name in enumerate(x_names):
    val_min = result_minimal.outcome_params[i]
    val_full = result_ml.outcome_params[i]
    diff = val_min - val_full
    print(f"{name:<18} {val_min:>16.4f} {val_full:>16.4f} {diff:>12.4f}")

# Selection parameters comparison
print("\n--- Selection Parameters ---")
print(f"{'Parameter':<18} {'Minimal (1 excl)':>16} {'Full (4 excl)':>16} {'Difference':>12}")
print("-" * 64)
print(f"{'sigma':<18} {result_minimal.sigma:>16.4f} {result_ml.sigma:>16.4f} "
      f"{result_minimal.sigma - result_ml.sigma:>12.4f}")
print(f"{'rho':<18} {result_minimal.rho:>16.4f} {result_ml.rho:>16.4f} "
      f"{result_minimal.rho - result_ml.rho:>12.4f}")
lambda_min = result_minimal.rho * result_minimal.sigma
lambda_full = result_ml.rho * result_ml.sigma
print(f"{'lambda (rho*sigma)':<18} {lambda_min:>16.4f} {lambda_full:>16.4f} "
      f"{lambda_min - lambda_full:>12.4f}")

# Log-likelihoods
print("\n--- Model Fit ---")
if result_minimal.llf is not None:
    print(f"{'Log-likelihood':<18} {result_minimal.llf:>16.4f} {result_ml.llf:>16.4f}")
    print(f"{'Converged':<18} {str(result_minimal.converged):>16} {str(result_ml.converged):>16}")

# Selection equation coefficients
print("\n--- Selection Equation ---")
print("\n  Minimal specification:")
for name, coef in zip(z_names_minimal, result_minimal.probit_params):
    print(f"    {name:<20} {coef:>12.4f}")

print("\n  Full specification:")
for name, coef in zip(z_names, result_ml.probit_params):
    print(f"    {name:<20} {coef:>12.4f}")

print("\n" + "=" * 75)

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Outcome coefficients
x_pos = np.arange(len(x_names))
width = 0.35

axes[0].bar(x_pos - width/2, result_minimal.outcome_params, width,
            label='Minimal (1 exclusion)', color='#f39c12', alpha=0.8, edgecolor='black')
axes[0].bar(x_pos + width/2, result_ml.outcome_params, width,
            label='Full (4 exclusions)', color='#2980b9', alpha=0.8, edgecolor='black')
axes[0].set_xlabel('Variable')
axes[0].set_ylabel('Coefficient')
axes[0].set_title('Outcome Equation Coefficients', fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(x_names, rotation=30, ha='right')
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].axhline(y=0, color='black', linewidth=0.8)

# Panel 2: Selection parameters
sel_names = ['sigma', 'rho', 'lambda']
sel_min = [result_minimal.sigma, result_minimal.rho, lambda_min]
sel_full = [result_ml.sigma, result_ml.rho, lambda_full]

x_pos2 = np.arange(len(sel_names))
axes[1].bar(x_pos2 - width/2, sel_min, width,
            label='Minimal (1 exclusion)', color='#f39c12', alpha=0.8, edgecolor='black')
axes[1].bar(x_pos2 + width/2, sel_full, width,
            label='Full (4 exclusions)', color='#2980b9', alpha=0.8, edgecolor='black')
axes[1].set_xlabel('Parameter')
axes[1].set_ylabel('Value')
axes[1].set_title('Selection Parameters', fontweight='bold')
axes[1].set_xticks(x_pos2)
axes[1].set_xticklabels(sel_names)
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axhline(y=0, color='black', linewidth=0.8)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex2_exclusion_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# -------------------------------------------------------
# Discussion
# -------------------------------------------------------
rho_diff = abs(result_minimal.rho - result_ml.rho)
beta_educ_diff = abs(result_minimal.outcome_params[1] - result_ml.outcome_params[1])

print("=" * 65)
print("  DISCUSSION: Sensitivity to Exclusion Restrictions")
print("=" * 65)
print(f"\n  Difference in rho:            {rho_diff:.4f}")
print(f"  Difference in beta(education): {beta_educ_diff:.4f}")

print("\n  Key findings:")
if rho_diff < 0.1:
    print("  - rho estimates are fairly stable across exclusion specifications.")
    print("    This suggests the selection correction is not overly sensitive")
    print("    to the specific choice of instruments.")
else:
    print("  - rho estimates differ meaningfully between specifications.")
    print("    This sensitivity may indicate weak identification with fewer")
    print("    exclusion restrictions, or that additional exclusions provide")
    print("    important identifying variation.")

print("\n  - Having more exclusion restrictions provides stronger")
print("    identification of the selection equation, reducing reliance")
print("    on functional form (the nonlinearity of the probit model).")
print("  - Variables like children_lt6 and age are plausibly excluded")
print("    from the wage equation (they affect participation but not")
print("    wages directly), strengthening the exclusion restriction.")
print("  - Reporting results across specifications is good practice.")
print("=" * 65)

---

## Exercise 3: Convergence Diagnostics (Medium)

**Task**: Explore the sensitivity of MLE convergence to starting values.

1. Estimate with the default warm start (from two-step)
2. Perturb starting values with small noise (10% of two-step estimates)
3. Perturb starting values with large noise (50% of two-step estimates)
4. Check if all converge to the same estimates

In [ ]:
# ============================================================
# EXERCISE 3 SOLUTION: Convergence Diagnostics
# ============================================================

# Step 1: Construct the default starting values from two-step
k_outcome = X.shape[1]
k_selection = Z.shape[1]

init_params_default = np.concatenate([
    result_2s.outcome_params,
    result_2s.probit_params,
    [np.log(result_2s.sigma)],
    [np.arctanh(np.clip(result_2s.rho, -0.99, 0.99))]
])

print(f"Number of parameters: {len(init_params_default)}")
print(f"  - Outcome equation (beta):   {k_outcome}")
print(f"  - Selection equation (gamma): {k_selection}")
print(f"  - log(sigma):                1")
print(f"  - arctanh(rho):              1")
print(f"\nDefault starting values (from two-step):")
print(f"  {init_params_default}")

In [ ]:
# Step 2: Optimize from three different starting points

# Use the model object's _log_likelihood method (returns negative LL for minimization)

# --- Default warm start ---
opt_default = minimize(
    model_ml._log_likelihood,
    init_params_default,
    method='BFGS',
    options={'maxiter': 1000}
)
print("=== Default Warm Start ===")
print(f"  Converged: {opt_default.success}")
print(f"  Neg log-lik: {opt_default.fun:.6f}")
print(f"  Iterations: {opt_default.nit}")

# --- Small perturbation (10%) ---
np.random.seed(123)
init_params_small = init_params_default * (1 + 0.1 * np.random.randn(len(init_params_default)))

opt_small = minimize(
    model_ml._log_likelihood,
    init_params_small,
    method='BFGS',
    options={'maxiter': 1000}
)
print("\n=== Small Perturbation (10%) ===")
print(f"  Converged: {opt_small.success}")
print(f"  Neg log-lik: {opt_small.fun:.6f}")
print(f"  Iterations: {opt_small.nit}")

# --- Large perturbation (50%) ---
np.random.seed(456)
init_params_large = init_params_default * (1 + 0.5 * np.random.randn(len(init_params_default)))

opt_large = minimize(
    model_ml._log_likelihood,
    init_params_large,
    method='BFGS',
    options={'maxiter': 1000}
)
print("\n=== Large Perturbation (50%) ===")
print(f"  Converged: {opt_large.success}")
print(f"  Neg log-lik: {opt_large.fun:.6f}")
print(f"  Iterations: {opt_large.nit}")

In [ ]:
# Step 3: Compare final parameter estimates

# Helper function to extract readable parameters from raw vector
def extract_params(raw_params):
    beta = raw_params[:k_outcome]
    gamma = raw_params[k_outcome:k_outcome + k_selection]
    sigma = np.exp(raw_params[-2])
    rho = np.tanh(raw_params[-1])
    return beta, gamma, sigma, rho

beta_def, gamma_def, sigma_def, rho_def = extract_params(opt_default.x)
beta_sml, gamma_sml, sigma_sml, rho_sml = extract_params(opt_small.x)
beta_lrg, gamma_lrg, sigma_lrg, rho_lrg = extract_params(opt_large.x)

print("=" * 80)
print("  CONVERGENCE COMPARISON: FINAL PARAMETER ESTIMATES")
print("=" * 80)

# Outcome equation
print("\n--- Outcome Equation (beta) ---")
print(f"{'Variable':<16} {'Default':>12} {'Small Pert':>12} {'Large Pert':>12} {'Max Diff':>12}")
print("-" * 66)
for i, name in enumerate(x_names):
    vals = [beta_def[i], beta_sml[i], beta_lrg[i]]
    max_diff = max(vals) - min(vals)
    print(f"{name:<16} {beta_def[i]:>12.4f} {beta_sml[i]:>12.4f} {beta_lrg[i]:>12.4f} {max_diff:>12.6f}")

# Selection equation
print("\n--- Selection Equation (gamma) ---")
print(f"{'Variable':<16} {'Default':>12} {'Small Pert':>12} {'Large Pert':>12} {'Max Diff':>12}")
print("-" * 66)
for i, name in enumerate(z_names):
    vals = [gamma_def[i], gamma_sml[i], gamma_lrg[i]]
    max_diff = max(vals) - min(vals)
    print(f"{name:<16} {gamma_def[i]:>12.4f} {gamma_sml[i]:>12.4f} {gamma_lrg[i]:>12.4f} {max_diff:>12.6f}")

# Selection parameters
print("\n--- Selection Parameters ---")
print(f"{'Parameter':<16} {'Default':>12} {'Small Pert':>12} {'Large Pert':>12} {'Max Diff':>12}")
print("-" * 66)
print(f"{'sigma':<16} {sigma_def:>12.4f} {sigma_sml:>12.4f} {sigma_lrg:>12.4f} "
      f"{max(sigma_def, sigma_sml, sigma_lrg) - min(sigma_def, sigma_sml, sigma_lrg):>12.6f}")
print(f"{'rho':<16} {rho_def:>12.4f} {rho_sml:>12.4f} {rho_lrg:>12.4f} "
      f"{max(rho_def, rho_sml, rho_lrg) - min(rho_def, rho_sml, rho_lrg):>12.6f}")

# Log-likelihood comparison
print("\n--- Log-Likelihood ---")
print(f"  Default:        {-opt_default.fun:.6f}")
print(f"  Small pert:     {-opt_small.fun:.6f}")
print(f"  Large pert:     {-opt_large.fun:.6f}")
print(f"  Max difference: {max(-opt_default.fun, -opt_small.fun, -opt_large.fun) - min(-opt_default.fun, -opt_small.fun, -opt_large.fun):.6f}")
print("=" * 80)

In [ ]:
# Visualize convergence comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Outcome coefficients across starting values
x_pos = np.arange(len(x_names))
width = 0.25

axes[0].bar(x_pos - width, beta_def, width, label='Default',
            color='#2980b9', alpha=0.8, edgecolor='black')
axes[0].bar(x_pos, beta_sml, width, label='Small Pert',
            color='#27ae60', alpha=0.8, edgecolor='black')
axes[0].bar(x_pos + width, beta_lrg, width, label='Large Pert',
            color='#e74c3c', alpha=0.8, edgecolor='black')
axes[0].set_xlabel('Variable')
axes[0].set_ylabel('Coefficient')
axes[0].set_title('Outcome Coefficients by Starting Point', fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(x_names, rotation=30, ha='right')
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].axhline(y=0, color='black', linewidth=0.8)

# Panel 2: sigma and rho
sel_names_plot = ['sigma', 'rho']
sel_default = [sigma_def, rho_def]
sel_small = [sigma_sml, rho_sml]
sel_large = [sigma_lrg, rho_lrg]

x_pos2 = np.arange(len(sel_names_plot))
axes[1].bar(x_pos2 - width, sel_default, width, label='Default',
            color='#2980b9', alpha=0.8, edgecolor='black')
axes[1].bar(x_pos2, sel_small, width, label='Small Pert',
            color='#27ae60', alpha=0.8, edgecolor='black')
axes[1].bar(x_pos2 + width, sel_large, width, label='Large Pert',
            color='#e74c3c', alpha=0.8, edgecolor='black')
axes[1].set_xlabel('Parameter')
axes[1].set_ylabel('Value')
axes[1].set_title('Selection Parameters by Starting Point', fontweight='bold')
axes[1].set_xticks(x_pos2)
axes[1].set_xticklabels(sel_names_plot)
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axhline(y=0, color='black', linewidth=0.8)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex3_convergence_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# -------------------------------------------------------
# Discussion
# -------------------------------------------------------
all_converged = opt_default.success and opt_small.success and opt_large.success
ll_vals = [-opt_default.fun, -opt_small.fun, -opt_large.fun]
ll_range = max(ll_vals) - min(ll_vals)

# Overall max parameter difference
all_params = np.vstack([opt_default.x, opt_small.x, opt_large.x])
max_param_range = np.max(np.ptp(all_params, axis=0))

print("=" * 65)
print("  DISCUSSION: Convergence Diagnostics")
print("=" * 65)
print(f"\n  All converged: {all_converged}")
print(f"  Log-likelihood range: {ll_range:.6f}")
print(f"  Max parameter range:  {max_param_range:.6f}")

print("\n  Interpretation:")
if ll_range < 0.01 and max_param_range < 0.1:
    print("  - All three starting points converge to essentially the same")
    print("    estimates. This indicates a well-behaved, unimodal")
    print("    log-likelihood surface near the optimum.")
    print("  - The warm-start strategy works well: even with large")
    print("    perturbations, the optimizer finds the same maximum.")
elif ll_range < 1.0:
    print("  - Small differences in final estimates suggest minor numerical")
    print("    sensitivity but all converge to similar regions.")
    print("  - The log-likelihood surface appears locally well-behaved.")
else:
    print("  - Non-trivial differences suggest the log-likelihood surface")
    print("    may have multiple local optima or flat regions.")
    print("  - The warm-start strategy is especially important here.")

print("\n  Practical lessons:")
print("  1. Always use the two-step warm start for MLE optimization.")
print("  2. Running from multiple starting points is a useful diagnostic.")
print("  3. If results differ across starting points, the model may be")
print("     weakly identified or the data may not support joint estimation.")
print("=" * 65)

---

## Exercise 4: Monte Carlo Comparison (Hard)

**Task**: Conduct a Monte Carlo simulation to compare finite-sample properties of two-step and MLE.

True DGP:
- Outcome: $y_i^* = \beta_0 + \beta_1 x_i + \varepsilon_i$ with $\beta = [1, 0.5]$
- Selection: $s_i^* = \gamma_0 + \gamma_1 x_i + \gamma_2 z_i + u_i$ with $\gamma = [0.3, 0.8, -0.5]$
- $(\varepsilon_i, u_i) \sim N(0, \Sigma)$ where $\sigma = 2$, $\rho = -0.5$
- $s_i = 1[s_i^* > 0]$; $y_i$ observed only if $s_i = 1$

Compare bias, RMSE, and coverage across 100 replications.

In [ ]:
# ============================================================
# EXERCISE 4 SOLUTION: Monte Carlo Comparison
# ============================================================

# True parameters
beta_true = np.array([1.0, 0.5])   # [intercept, slope]
gamma_true = np.array([0.3, 0.8, -0.5])  # [intercept, x coef, z coef (exclusion)]
sigma_true = 2.0
rho_true = -0.5

n_obs = 500
n_reps = 100

# Covariance matrix for (epsilon, u)
# Var(epsilon) = sigma^2, Var(u) = 1 (normalized), Cov = rho * sigma
cov_matrix = np.array([
    [sigma_true**2,       rho_true * sigma_true],
    [rho_true * sigma_true, 1.0]
])

print("True DGP Parameters:")
print(f"  beta  = {beta_true}")
print(f"  gamma = {gamma_true}")
print(f"  sigma = {sigma_true}")
print(f"  rho   = {rho_true}")
print(f"\nCovariance matrix of (epsilon, u):")
print(f"  {cov_matrix}")
print(f"\nn_obs = {n_obs}, n_reps = {n_reps}")

In [ ]:
# Run the Monte Carlo simulation
np.random.seed(42)

# Storage for estimates
results_mc = {
    'beta0_2s': [], 'beta1_2s': [], 'sigma_2s': [], 'rho_2s': [],
    'beta0_ml': [], 'beta1_ml': [], 'sigma_ml': [], 'rho_ml': [],
    'converged_ml': [],
    'selection_rate': [],
}

n_failures = 0

for rep in range(n_reps):
    if (rep + 1) % 20 == 0:
        print(f"  Replication {rep + 1}/{n_reps}...")

    # Generate regressors
    x_i = np.random.randn(n_obs)         # Shared regressor
    z_i = np.random.randn(n_obs)         # Exclusion restriction (only in selection)

    # Generate correlated errors
    errors = np.random.multivariate_normal(
        mean=[0, 0], cov=cov_matrix, size=n_obs
    )
    epsilon_i = errors[:, 0]  # Outcome error (Var = sigma^2)
    u_i = errors[:, 1]        # Selection error (Var = 1)

    # Build outcome and selection regressors with constant
    X_mc = sm.add_constant(x_i)
    Z_mc = sm.add_constant(np.column_stack([x_i, z_i]))

    # Latent outcome
    y_star = X_mc @ beta_true + epsilon_i

    # Selection
    s_star = Z_mc @ gamma_true + u_i
    s_i = (s_star > 0).astype(float)

    # Observed outcome (0 for non-selected)
    y_obs = y_star * s_i

    sel_rate = s_i.mean()
    results_mc['selection_rate'].append(sel_rate)

    # Skip if degenerate selection
    if sel_rate < 0.05 or sel_rate > 0.95:
        n_failures += 1
        continue

    try:
        # Two-step estimation
        m_2s = PanelHeckman(
            endog=y_obs, exog=X_mc, selection=s_i,
            exog_selection=Z_mc, method='two_step'
        )
        r_2s = m_2s.fit()

        results_mc['beta0_2s'].append(r_2s.outcome_params[0])
        results_mc['beta1_2s'].append(r_2s.outcome_params[1])
        results_mc['sigma_2s'].append(r_2s.sigma)
        results_mc['rho_2s'].append(r_2s.rho)

        # MLE estimation
        m_ml = PanelHeckman(
            endog=y_obs, exog=X_mc, selection=s_i,
            exog_selection=Z_mc, method='mle'
        )
        r_ml = m_ml.fit()

        results_mc['beta0_ml'].append(r_ml.outcome_params[0])
        results_mc['beta1_ml'].append(r_ml.outcome_params[1])
        results_mc['sigma_ml'].append(r_ml.sigma)
        results_mc['rho_ml'].append(r_ml.rho)
        results_mc['converged_ml'].append(r_ml.converged)

    except Exception as e:
        n_failures += 1
        continue

n_successful = len(results_mc['beta0_2s'])
print(f"\nCompleted: {n_successful} successful replications")
print(f"Failures:  {n_failures}")
print(f"MLE convergence rate: {np.mean(results_mc['converged_ml']):.1%}")
print(f"Mean selection rate: {np.mean(results_mc['selection_rate']):.2%}")

In [ ]:
# Convert to arrays for analysis
mc = {k: np.array(v) for k, v in results_mc.items()}

# -------------------------------------------------------
# Compute Bias and RMSE
# -------------------------------------------------------
def compute_mc_stats(estimates, true_value, label):
    """Compute Monte Carlo bias, RMSE, and std dev."""
    bias = np.mean(estimates) - true_value
    rmse = np.sqrt(np.mean((estimates - true_value) ** 2))
    std = np.std(estimates)
    median = np.median(estimates)
    return {
        'Parameter': label,
        'True': true_value,
        'Mean': np.mean(estimates),
        'Median': median,
        'Bias': bias,
        'Std': std,
        'RMSE': rmse,
    }

# Two-step statistics
stats_2s = [
    compute_mc_stats(mc['beta0_2s'], beta_true[0], 'beta_0'),
    compute_mc_stats(mc['beta1_2s'], beta_true[1], 'beta_1'),
    compute_mc_stats(mc['sigma_2s'], sigma_true, 'sigma'),
    compute_mc_stats(mc['rho_2s'], rho_true, 'rho'),
]

# MLE statistics
stats_ml = [
    compute_mc_stats(mc['beta0_ml'], beta_true[0], 'beta_0'),
    compute_mc_stats(mc['beta1_ml'], beta_true[1], 'beta_1'),
    compute_mc_stats(mc['sigma_ml'], sigma_true, 'sigma'),
    compute_mc_stats(mc['rho_ml'], rho_true, 'rho'),
]

df_2s = pd.DataFrame(stats_2s).set_index('Parameter')
df_ml = pd.DataFrame(stats_ml).set_index('Parameter')

print("=" * 75)
print("  MONTE CARLO RESULTS: TWO-STEP ESTIMATOR")
print("=" * 75)
print(df_2s.to_string(float_format=lambda x: f"{x:.4f}"))

print("\n" + "=" * 75)
print("  MONTE CARLO RESULTS: MLE ESTIMATOR")
print("=" * 75)
print(df_ml.to_string(float_format=lambda x: f"{x:.4f}"))

In [ ]:
# -------------------------------------------------------
# Head-to-head comparison table
# -------------------------------------------------------
print("\n" + "=" * 80)
print("  HEAD-TO-HEAD COMPARISON: TWO-STEP vs MLE")
print("=" * 80)
print(f"\n{'Parameter':<12} {'True':>8} | {'Bias_2S':>10} {'Bias_ML':>10} | "
      f"{'RMSE_2S':>10} {'RMSE_ML':>10} | {'Winner':>8}")
print("-" * 80)

param_names = ['beta_0', 'beta_1', 'sigma', 'rho']
true_vals = [beta_true[0], beta_true[1], sigma_true, rho_true]

for i, (name, true_val) in enumerate(zip(param_names, true_vals)):
    bias_2s = df_2s.loc[name, 'Bias']
    bias_ml = df_ml.loc[name, 'Bias']
    rmse_2s = df_2s.loc[name, 'RMSE']
    rmse_ml = df_ml.loc[name, 'RMSE']
    winner = 'MLE' if rmse_ml < rmse_2s else '2-Step'
    print(f"{name:<12} {true_val:>8.2f} | {bias_2s:>10.4f} {bias_ml:>10.4f} | "
          f"{rmse_2s:>10.4f} {rmse_ml:>10.4f} | {winner:>8}")

print("\nNote: 'Winner' is determined by lower RMSE.")
print("=" * 80)

In [ ]:
# -------------------------------------------------------
# Coverage analysis (approximate 95% CI)
# -------------------------------------------------------
# For MLE, use the empirical standard deviation as an approximation
# of the standard error. In practice one would use the Hessian-based
# SEs from each replication, but here we use the MC std as a proxy.

print("=" * 70)
print("  COVERAGE ANALYSIS (Approximate 95% Confidence Intervals)")
print("=" * 70)
print("\n  Using +/- 1.96 * (empirical SE from MC replications) as the CI.")
print("  Nominal coverage should be approximately 95%.\n")

param_keys_2s = ['beta0_2s', 'beta1_2s', 'sigma_2s', 'rho_2s']
param_keys_ml = ['beta0_ml', 'beta1_ml', 'sigma_ml', 'rho_ml']

print(f"{'Parameter':<12} {'True':>8} | {'Coverage_2S':>12} {'Coverage_ML':>12}")
print("-" * 56)

for name, true_val, key_2s, key_ml in zip(param_names, true_vals, param_keys_2s, param_keys_ml):
    est_2s = mc[key_2s]
    est_ml = mc[key_ml]

    # Empirical SE from MC distribution
    se_2s = np.std(est_2s)
    se_ml = np.std(est_ml)

    # Coverage: fraction of replications where true value falls
    # within [estimate - 1.96*SE, estimate + 1.96*SE]
    cover_2s = np.mean(np.abs(est_2s - true_val) <= 1.96 * se_2s)
    cover_ml = np.mean(np.abs(est_ml - true_val) <= 1.96 * se_ml)

    print(f"{name:<12} {true_val:>8.2f} | {cover_2s:>11.1%} {cover_ml:>11.1%}")

print("\n  Note: These are approximate coverage rates using the empirical")
print("  MC standard deviation. Proper coverage requires replication-level")
print("  standard errors (e.g., from the inverse Hessian for MLE).")
print("=" * 70)

In [ ]:
# -------------------------------------------------------
# Visualization: sampling distributions
# -------------------------------------------------------
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

plot_configs = [
    ('beta0', beta_true[0], r'$\beta_0$ (intercept)'),
    ('beta1', beta_true[1], r'$\beta_1$ (slope)'),
    ('sigma', sigma_true, r'$\sigma$'),
    ('rho', rho_true, r'$\rho$'),
]

for ax, (pname, true_val, label) in zip(axes.flat, plot_configs):
    est_2s = mc[f'{pname}_2s']
    est_ml = mc[f'{pname}_ml']

    ax.hist(est_2s, bins=25, alpha=0.5, color='#3498db',
            label='Two-Step', density=True, edgecolor='black', linewidth=0.5)
    ax.hist(est_ml, bins=25, alpha=0.5, color='#e74c3c',
            label='MLE', density=True, edgecolor='black', linewidth=0.5)
    ax.axvline(x=true_val, color='black', linewidth=2.5, linestyle='--',
               label=f'True = {true_val}')
    ax.axvline(x=np.mean(est_2s), color='#2980b9', linewidth=1.5, linestyle=':')
    ax.axvline(x=np.mean(est_ml), color='#c0392b', linewidth=1.5, linestyle=':')

    ax.set_xlabel(label, fontsize=12)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'Sampling Distribution of {label}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex4_monte_carlo_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Bar chart comparing bias and RMSE
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x_pos = np.arange(len(param_names))
width = 0.35

# Panel 1: Absolute Bias
bias_2s_vals = [abs(df_2s.loc[p, 'Bias']) for p in param_names]
bias_ml_vals = [abs(df_ml.loc[p, 'Bias']) for p in param_names]

axes[0].bar(x_pos - width/2, bias_2s_vals, width,
            label='Two-Step', color='#3498db', alpha=0.8, edgecolor='black')
axes[0].bar(x_pos + width/2, bias_ml_vals, width,
            label='MLE', color='#e74c3c', alpha=0.8, edgecolor='black')
axes[0].set_xlabel('Parameter')
axes[0].set_ylabel('|Bias|')
axes[0].set_title('Absolute Bias Comparison', fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(param_names)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Panel 2: RMSE
rmse_2s_vals = [df_2s.loc[p, 'RMSE'] for p in param_names]
rmse_ml_vals = [df_ml.loc[p, 'RMSE'] for p in param_names]

axes[1].bar(x_pos - width/2, rmse_2s_vals, width,
            label='Two-Step', color='#3498db', alpha=0.8, edgecolor='black')
axes[1].bar(x_pos + width/2, rmse_ml_vals, width,
            label='MLE', color='#e74c3c', alpha=0.8, edgecolor='black')
axes[1].set_xlabel('Parameter')
axes[1].set_ylabel('RMSE')
axes[1].set_title('RMSE Comparison', fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(param_names)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex4_bias_rmse_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# -------------------------------------------------------
# Discussion
# -------------------------------------------------------
print("=" * 70)
print("  DISCUSSION: Monte Carlo Findings")
print("=" * 70)

# Count how many parameters MLE wins on RMSE
mle_wins = sum(
    1 for p in param_names
    if df_ml.loc[p, 'RMSE'] < df_2s.loc[p, 'RMSE']
)

print(f"\n  MLE has lower RMSE for {mle_wins}/{len(param_names)} parameters.")
print(f"  MLE convergence rate: {np.mean(mc['converged_ml']):.1%}")
print(f"  Successful replications: {n_successful}/{n_reps}")

print("\n  Key findings:")
print("  1. Both estimators are consistent: as n grows, bias shrinks toward 0.")
print("  2. MLE typically achieves lower RMSE, especially for sigma and rho,")
print("     because it uses the full information from the joint likelihood.")
print("  3. Two-step is more robust: it always 'converges' (no iterative")
print("     optimization), while MLE can occasionally fail.")
print("  4. The efficiency advantage of MLE is most visible for the selection")
print("     parameters (rho, sigma), which are directly estimated from the")
print("     joint likelihood rather than backed out from auxiliary regressions.")
print("  5. With n=500 observations, the efficiency gains are meaningful but")
print("     not dramatic. The advantage grows with larger samples.")

print("\n  Practical recommendations:")
print("  - Use two-step for initial exploration and robustness checks.")
print("  - Use MLE for final results when bivariate normality is plausible.")
print("  - Always report both as a sensitivity check.")
print("=" * 70)

---

## Summary

This solution notebook demonstrated:

1. **Exercise 1**: Manual computation of the Heckman log-likelihood for selected and non-selected observations, verifying that the sum matches the model output.

2. **Exercise 2**: Sensitivity analysis showing how the number and quality of exclusion restrictions affects the estimated selection parameters (rho, sigma) and outcome coefficients.

3. **Exercise 3**: Convergence diagnostics confirming that the warm-start strategy (from two-step) produces a reliable optimum, even when starting values are perturbed.

4. **Exercise 4**: A Monte Carlo simulation comparing the finite-sample properties (bias, RMSE, coverage) of two-step and MLE estimation, showing the efficiency advantage of MLE under correct specification.